In [1]:
import os
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader, random_split

In [2]:
# Titanic 데이터셋을 위한 Dataset 클래스 정의
class TitanicDataset(Dataset):
    def __init__(self, X, y):
        # X와 y는 각각 입력 데이터와 타겟 레이블
        self.X = torch.FloatTensor(X)  # 입력 데이터를 torch의 FloatTensor로 변환
        self.y = torch.LongTensor(y)  # 타겟 데이터를 torch의 LongTensor로 변환

    def __len__(self):
        # 데이터셋의 크기를 반환
        return len(self.X)

    def __getitem__(self, idx):
        # 주어진 인덱스에 해당하는 입력 데이터와 타겟을 반환
        feature = self.X[idx]
        target = self.y[idx]
        return {'input': feature, 'target': target}

    def __str__(self):
        # 데이터셋의 정보(크기와 입력/타겟의 형태)를 반환
        return "Data Size: {0}, Input Shape: {1}, Target Shape: {2}".format(
            len(self.X), self.X.shape, self.y.shape
        )

In [3]:
# 테스트 데이터셋을 위한 Dataset 클래스 정의 (타겟이 없음)
class TitanicTestDataset(Dataset):
    def __init__(self, X):
        # 입력 데이터만 받음
        self.X = torch.FloatTensor(X)  # 입력 데이터를 torch의 FloatTensor로 변환

    def __len__(self):
        # 데이터셋의 크기를 반환
        return len(self.X)

    def __getitem__(self, idx):
        # 주어진 인덱스에 해당하는 입력 데이터를 반환 (타겟 없음)
        feature = self.X[idx]
        return {'input': feature}

    def __str__(self):
        # 데이터셋의 정보(크기와 입력의 형태)를 반환
        return "Data Size: {0}, Input Shape: {1}".format(len(self.X), self.X.shape)

In [4]:
# 데이터셋을 불러오고 전처리하는 함수
def get_preprocessed_dataset():
    # 현재 파일 경로를 수동으로 설정 (Jupyter Notebook에서는 __file__을 사용할 수 없음)
    CURRENT_FILE_PATH = os.path.abspath('')  # 데이터가 위치한 경로로 수정

    # 학습 데이터와 테스트 데이터 파일 경로 설정
    train_data_path = os.path.join(CURRENT_FILE_PATH, "train.csv")
    test_data_path = os.path.join(CURRENT_FILE_PATH, "test.csv")

    # CSV 파일을 데이터프레임으로 로드
    train_df = pd.read_csv(train_data_path)
    test_df = pd.read_csv(test_data_path)
    # 학습 데이터와 테스트 데이터를 하나의 데이터프레임으로 병합
    all_df = pd.concat([train_df, test_df], sort=False)

    # 데이터 전처리 함수들 적용
    all_df = get_preprocessed_dataset_1(all_df)
    all_df = get_preprocessed_dataset_2(all_df)
    all_df = get_preprocessed_dataset_3(all_df)
    all_df = get_preprocessed_dataset_4(all_df)
    all_df = get_preprocessed_dataset_5(all_df)
    all_df = get_preprocessed_dataset_6(all_df)

    # 학습 데이터의 입력과 타겟을 구분
    train_X = all_df[~all_df["Survived"].isnull()].drop("Survived", axis=1).reset_index(drop=True)  # 생존 데이터를 제외한 나머지 입력값
    train_y = train_df["Survived"]  # 생존 여부

    # 테스트 데이터의 입력만 구분 (타겟이 없음)
    test_X = all_df[all_df["Survived"].isnull()].drop("Survived", axis=1).reset_index(drop=True)

    # TitanicDataset 객체 생성 (학습 데이터셋)
    dataset = TitanicDataset(train_X.values, train_y.values)
    # 학습 데이터셋을 학습과 검증용으로 80:20 비율로 나눔
    train_dataset, validation_dataset = random_split(dataset, [0.8, 0.2])
    # 테스트 데이터셋 생성
    test_dataset = TitanicTestDataset(test_X.values)

    # 학습, 검증, 테스트 데이터셋 반환
    return train_dataset, validation_dataset, test_dataset

In [5]:
# 첫 번째 전처리 단계: Pclass별로 평균 Fare값을 계산하여 결측치 채움
def get_preprocessed_dataset_1(all_df):
    Fare_mean = all_df[["Pclass", "Fare"]].groupby("Pclass").mean().reset_index()
    Fare_mean.columns = ["Pclass", "Fare_mean"]
    all_df = pd.merge(all_df, Fare_mean, on="Pclass", how="left")
    all_df.loc[(all_df["Fare"].isnull()), "Fare"] = all_df["Fare_mean"]
    return all_df

In [6]:
# 두 번째 전처리 단계: Name 컬럼을 세 개의 부분으로 분리 (성, 경칭, 이름)
def get_preprocessed_dataset_2(all_df):
    name_df = all_df["Name"].str.split("[,.]", n=2, expand=True)
    name_df.columns = ["family_name", "honorific", "name"]
    name_df["family_name"] = name_df["family_name"].str.strip()
    name_df["honorific"] = name_df["honorific"].str.strip()
    name_df["name"] = name_df["name"].str.strip()
    all_df = pd.concat([all_df, name_df], axis=1)
    return all_df

In [7]:
# 세 번째 전처리 단계: 경칭별로 나이의 중간값을 계산하여 결측치 채움
def get_preprocessed_dataset_3(all_df):
    honorific_age_mean = all_df[["honorific", "Age"]].groupby("honorific").median().round().reset_index()
    honorific_age_mean.columns = ["honorific", "honorific_age_mean"]
    all_df = pd.merge(all_df, honorific_age_mean, on="honorific", how="left")
    all_df.loc[(all_df["Age"].isnull()), "Age"] = all_df["honorific_age_mean"]
    all_df = all_df.drop(["honorific_age_mean"], axis=1)
    return all_df

In [8]:
# 네 번째 전처리 단계: 가족 수(family_num)와 혼자 탑승 여부(alone) 컬럼 추가
def get_preprocessed_dataset_4(all_df):
    all_df["family_num"] = all_df["Parch"] + all_df["SibSp"]
    all_df.loc[all_df["family_num"] == 0, "alone"] = 1
    all_df["alone"].fillna(0, inplace=True)
    all_df = all_df.drop(["PassengerId", "Name", "family_name", "name", "Ticket", "Cabin"], axis=1)
    return all_df


In [9]:
# 다섯 번째 전처리 단계: honorific 값의 개수를 줄임
def get_preprocessed_dataset_5(all_df):
    all_df.loc[
        ~((all_df["honorific"] == "Mr") |
          (all_df["honorific"] == "Miss") |
          (all_df["honorific"] == "Mrs") |
          (all_df["honorific"] == "Master")),
        "honorific"
    ] = "other"
    all_df["Embarked"].fillna("missing", inplace=True)
    return all_df


In [10]:
# 여섯 번째 전처리 단계: 카테고리형 변수를 LabelEncoder를 사용해 수치값으로 변환
def get_preprocessed_dataset_6(all_df):
    category_features = all_df.columns[all_df.dtypes == "object"]
    from sklearn.preprocessing import LabelEncoder
    for category_feature in category_features:
        le = LabelEncoder()
        if all_df[category_feature].dtypes == "object":
            le = le.fit(all_df[category_feature])
            all_df[category_feature] = le.transform(all_df[category_feature])
    return all_df


In [11]:
# 모델 정의 (입력 차원과 출력 차원은 11과 2로 설정)
from torch import nn
class MyModel(nn.Module):
    def __init__(self, n_input, n_output):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(n_input, 30),  # 첫 번째 레이어 (11 입력, 30 유닛)
            nn.ReLU(),               # 활성화 함수 ReLU
            nn.Linear(30, 30),       # 두 번째 레이어
            nn.ReLU(),
            nn.Linear(30, n_output), # 출력 레이어 (30 유닛에서 2개의 출력)
        )

    def forward(self, x):
        return self.model(x)

In [12]:
# 테스트 데이터셋에 대해 모델을 평가하는 함수
def test(test_data_loader):
    print("[TEST]")
    batch = next(iter(test_data_loader))  # 배치 중 하나를 가져옴
    print("{0}".format(batch['input'].shape))
    my_model = MyModel(n_input=11, n_output=2)  # 모델 생성
    output_batch = my_model(batch['input'])     # 모델에 입력 데이터를 전달하여 예측 생성
    prediction_batch = torch.argmax(output_batch, dim=1)  # 예측에서 가장 높은 확률의 클래스를 선택
    for idx, prediction in enumerate(prediction_batch, start=892):
        print(idx, prediction.item())  # 예측 결과 출력

In [13]:

# 메인 실행 부분
if __name__ == "__main__":
    # 학습, 검증, 테스트 데이터셋 생성
    train_dataset, validation_dataset, test_dataset = get_preprocessed_dataset()

    print("train_dataset: {0}, validation_dataset.shape: {1}, test_dataset: {2}".format(
        len(train_dataset), len(validation_dataset), len(test_dataset)
    ))
    print("#" * 50, 1)

    # 학습 데이터셋에서 각 샘플의 입력과 타겟을 출력
    for idx, sample in enumerate(train_dataset):
        print("{0} - {1}: {2}".format(idx, sample['input'], sample['target']))

    print("#" * 50, 2)

    # DataLoader 생성 (학습, 검증, 테스트용)
    train_data_loader = DataLoader(dataset=train_dataset, batch_size=16, shuffle=True)
    validation_data_loader = DataLoader(dataset=validation_dataset, batch_size=16, shuffle=True)
    test_data_loader = DataLoader(dataset=test_dataset, batch_size=len(test_dataset))

    print("[TRAIN]")
    for idx, batch in enumerate(train_data_loader):
        print("{0} - {1}: {2}".format(idx, batch['input'].shape, batch['target'].shape))

    print("[VALIDATION]")
    for idx, batch in enumerate(validation_data_loader):
        print("{0} - {1}: {2}".format(idx, batch['input'].shape, batch['target'].shape))

    print("#" * 50, 3)

    # 테스트 함수 호출
    test(test_data_loader)

/tmp/ipykernel_11509/111853837.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  all_df["alone"].fillna(0, inplace=True)
/tmp/ipykernel_11509/4033177332.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using '

train_dataset: 713, validation_dataset.shape: 178, test_dataset: 418
################################################## 1
0 - tensor([ 3.0000,  0.0000, 26.0000,  0.0000,  0.0000,  7.9250,  2.0000, 13.3029,
         1.0000,  0.0000,  1.0000]): 1
1 - tensor([ 3.0000,  1.0000, 21.0000,  0.0000,  0.0000,  8.6625,  2.0000, 13.3029,
         2.0000,  0.0000,  1.0000]): 0
2 - tensor([ 1.0000,  1.0000, 29.0000,  0.0000,  0.0000, 30.5000,  2.0000, 87.5090,
         2.0000,  0.0000,  1.0000]): 1
3 - tensor([ 1.0000,  0.0000, 51.0000,  1.0000,  0.0000, 77.9583,  2.0000, 87.5090,
         3.0000,  1.0000,  0.0000]): 1
4 - tensor([ 3.0000,  1.0000, 19.0000,  0.0000,  0.0000,  7.6500,  2.0000, 13.3029,
         2.0000,  0.0000,  1.0000]): 0
5 - tensor([ 3.0000,  1.0000, 22.0000,  0.0000,  0.0000,  7.2500,  2.0000, 13.3029,
         2.0000,  0.0000,  1.0000]): 0
6 - tensor([ 3.0000,  0.0000, 36.0000,  1.0000,  0.0000, 16.1000,  2.0000, 13.3029,
         3.0000,  1.0000,  0.0000]): 1
7 - tensor([ 2.00